# Linear Models & Support Vector Machines

In this lesson we will explore a new way to think about classification. Rather than trying to directly classify new examples based on old ones to model the entire distribution of classes, we can simply model the *boundaries* between different classes.

First, we will gain some intuition around the benefits of modeling decision boundaries before exploring two different algorithms for boundary creation.

Next, we will explore the first of our boundary creation algorithms: Linear Discriminant Analysis (LDA). LDA will be our first exposure to representation learning; specifically classification via dimensionality reduction. LDA is a supervised algorithm, typically used for binary classification problems.

Finally, we will explore a method for boundary generation by finding the Max-Margin Classifier, also known as Support Vector Machines (SVM). SVM is a supervised algorithm, typically used for classification or regression problems.

## Decision Boundaries

We are going to shift our way of thinking about classification from directly classifying new examples based on old ones, to modeling a function that separates classes (ie. creating boundaries). The **discriminant** is a function that models decision boundaries between classes. In a bounded space, a novel data point will simply belong to the class it is bounded by. This bounded region is known as the **decision region**. In a multivariate dataset it can be difficult to visualize what a boundary represents, thus in order to understand where boundaries exist we must change the *representation* of the data without loss of information.

Consider the image below, with class distributions 1 (red), 2 (green), and 3 (blue). Data is distributed on a two-dimensional plane, and is separated into three distinct decision regions by purple discriminant lines. Notice the image on the left is divided by linear boundaries, while the image on the right is divided by non-linear boundaries. The image on the left is simple to model, but doesn't appear to be as accurate; for example, we see a significant number of red "1s" creeping into the rightmost boundary. The image on the right appears to correct the inaccuracies of the linear boundary for the most part, at the expense of being more difficult to model. In theory, using very complex, non-linear discriminants to approximate boundaries sounds nice, and can potentially separate boundaries perfectly. In practice, however, it often results in a **hypothesis space**, or set of all possible outputs, that is so large it becomes intractable. This example is trivial enough that a non-linear boundary for three classes is feasible, however, if we consider examples with many dimensions and many classes, then generating non-linear function boundaries becomes much harder to manage.

<img src="../img/3_discriminant.png" alt="http://uc-r.github.io/discriminant_analysis" style="width: 450px;"/>

### Mathematical Intuition

When we say "linear" we mean the separating boundary will be a **hyperplane**, or **decision surface**, which is a linear plane one dimension less than our original feature space. In two-dimensions this hyperplane is a line, but in higher dimensions we can still refer to this as a "linear" function. In lower dimensional spaces, the concept of a hyperplane is immediately obvious. However in much higher dimensions it becomes nearly impossible to conceptualize, let alone determine which boundary a data point exists in. Luckily, there is only one trick you need to know to determine which side of a hyperplane you are on. To gain intuition behind this, let's use an example in a low dimensional space.

Consider an input feature vector $\mathbf{x}\in\mathbb{R}^2$, and a line $L\in\mathbb{R}$ as our proposed boundary line. Our goal is to find a projection of $\mathbf{x}$ onto $L$, and determine a threshold to separate classes. We define $L$ as follows: $L=\{\text{c}\mathbf{w}|\text{c}\in\mathbb{R}\}$, where $\mathbf{w}$ is a **basis** for all vectors in $L$, hence we can say a linear combination of $\mathbf{w}$ uniquely represents all vectors in $L$. Let us define a hyperplane $P$ that is a plane *perpendicular* to $L$. Thus, $\mathbf{w}$ is **normal** to the hyperplane $P$, that is to say $\mathbf{w}$ is a unit vector perpendicular to the hyperplane. The projection of $\mathbf{x}$ onto $L$ is simply the **dot product** of $\mathbf{w}$ and $\mathbf{x}$, where $\mathbf{x}$ exists on $P$. The dot product is defined in \eqref{eq:dot_product}, where $x_i$, and $w_i$ are elements of vectors $\mathbf{x}$ and $\mathbf{w}$ respectively. Since we can represent a boundary by a vector $\mathbf{w}$, we can now treat the problem of finding a boundary line as one of estimating elements of $\mathbf{w}$, so there will be a single parameter for each dimension of the vector (same as the dimensionality of the data).

\begin{equation}
\mathbf{w}\cdot\mathbf{x}=\sum_{i=1}^n{w_ix_i}=w_1x_1+\cdots+w_nx_n
\label{eq:dot_product}\tag{3.1}
\end{equation}

\eqref{eq:dot_product} will encode each data point as a single scalar value on a line, which is effectively the distance from the origin along the vector. We can choose a scalar threshold value, $b$, such that projections belong to one class if $\mathbf{w}\cdot\mathbf{x}-b<0$, and the other class if $\mathbf{w}\cdot\mathbf{x}-b>0$.

If any data point can be correctly classified by this boundary, then we say the classes are **linearly separable**. If no hyperplane exists that can separate our classes correctly, we say our data is not linearly separable.

How do we choose how many boundaries exist for our dataset? Well, it is trivial to show a single boundary can separate two classes of data points. If we are presented with more than two classes we need multiple decision boundaries. The typical approach is to use a *one-vs-all* method, meaning there will be one decision boundary per class. This can lead to some problems, specifically where a new point lies in a region intersected by more than one hyperplane. There are several algorithms that exist to choose a hyperplane, most notably is through "gradient descent", which works by picking a random candidate split plane, and repeatedly making small adjustments to it in ways that reduce the error measure. While this level of specificity is beyond the scope of this lesson, if this subject is interesting to you I encourage you to read about techniques for generating "good" boundaries.

As we will see, finding the "best" boundary hyperplane is not trivial. There may be choices that give perfect accuracy on a training set, or there may be none. In either case, let's first explore a linear technique for generating boundaries.

## Linear Discriminant Analysis

Consider following data point distribution of two classes, red and blue:

<img src="../img/3_linearly_inseperable_data.png" alt="https://sthalles.github.io/fisher-linear-discriminant/" style="width: 350px;"/>

Clearly, there is no way to draw a line to separate the red and blue points and achieve good predictive results, as seen in the left hand image below. Instead, we can transform the data in such a way that we can separate classes with a line, as seen in the right hand image below. In this case, a new linearly separable representation of the distribution is found by squaring the input vectors. How did we know to square the input feature vectors to make this problem linearly separable? The answer to this is not trivial, and involves learning a new *representation*, which we will cover in more depth in the "Deep Learning" lessons.

<img src="../img/3_feature_transformation.png" alt="https://sthalles.github.io/fisher-linear-discriminant/" style="width: 500px;"/>

Linear Discriminant Analysis (LDA), most commonly implemented by **Fisher’s Linear Discriminant**, transforms data into a lower dimensional representation of the original space. The action of reducing the dimensionality of a dataset is known as **dimensionality reduction**. Dimensionality reduction will be covered in much greater detail in future lessons, but for now we will use it as a tool to create boundaries in datasets. We attempt to find a representation of our data samples by projecting the samples onto a plane of one less dimension. For example, if we have a plot of two-dimensional vectors, we would like to project them onto a line, which is a linear boundary in one-dimensional space. You can effectively run the LDA repeatedly as a form of dimensionality reduction for higher dimensional datasets, but this is not advised when solving classification problems. Typically, you want to project your data into a space one dimension less, then apply some other classification algorithm to the projected data.

Fisher wanted to maximize a function that returns a large variance among classes, and a small variance within classes. In other words, a function that, when maximized, increases the distance between different classes, and reduces the distance within same class data points. Recall our issue stated in the Introduction section, where a linear separation is not accurate, since there can be classes of a different label belonging to another label's decision region. LDA attempts to resolve this issue by finding a *distinct* linear boundary via changing the representation of the original data space. 

It should be noted, however, that it is possible LDA won't find a linearly separable boundary between classes that gives reasonable accuracy. In fact, if the projected mean values of two classes are equal then it is impossible for LDA to find a new hyperplane that makes both classes linearly separable. We are simply gaining another method to generalize a rule for classification, and it is possible this method will not work for certain data distributions.

### Mathematical Intuition

For the sake of example, we will consider a binary classification problem similar to the data distribution shown in the image below. As you can see, each data point belongs to one of two class labels, $G$ and $B$, with features in $\mathbb{R}^2$, and a projection space in $\mathbb{R}$. The proposed boundary is defined by the red line, which is simply a function of the vector normal to the hyperplane, will be estimated via Fisher's Linear Discriminant.

<img src="../img/3_lda.jpg" alt="https://www.geeksforgeeks.org/ml-linear-discriminant-analysis/" style="width: 350px;"/>

So how does LDA help us find the "best" hyperplane configuration? Fisher outlines the following two criteria:
1. Maximizing the distance between mean projected values of different classes. ($\hat{m}$)
2. Minimizing the **scatter**, or the "within-class" variance in the projected space, within each class. ($\hat{s}$)

Formally, LDA aims to *maximize* the objective function \eqref{eq:fisher}, which is the ratio of the *absolute* difference between projected means, normalized by the sum of projected within-class scatter. Intuitively, by maximizing the distance between different classes, and minimizing how spread out vectors in the same class are, we can define a hyperplane that splits different classes well.

\begin{equation}
J(\mathbf{w})=\frac{(\hat{m_G}-\hat{m_B})^2}{\hat{s_G}+\hat{s_B}}
\label{eq:fisher}\tag{3.2}
\end{equation}

Subscripts $G$ and $B$ represent the projected means, and projected scatter of class for classes $G$, and $B$ respectively. $J(\mathbf{w})$ is simply a function to estimate elements along the boundary vector. Let's dig a little deeper into what LDA is actually doing.

As seen in the previous section, using the dot product of $\mathbf{x}$ and $\mathbf{w}$ can find the represented scalar projection, $y$, on our boundary line. Likewise, the **transpose** of $\mathbf{w}$, denoted by $\mathbf{w}^\intercal$, multiplied by $\mathbf{x}$ achieves the same result. The transpose of a $n\times 1$ column-vector is simply a $1\times n$ row-vector, where column values are mapped to row values. We can say $y=\mathbf{w}\cdot\mathbf{x}$ is equivalent to \eqref{eq:linear}. Visually, the orthogonal projection is represented by the dotted lines mapping feature vectors to the hyperplane line. Formally, Fisher's Linear Discriminant projects vector $\mathbf{x}$ from our input space in $\mathbb{R}^2$ onto $y$, a scalar value in $\mathbb{R}$ by some linear combination of $\mathbf{w}^\intercal$ and $\mathbf{x}$.

\begin{equation}
y=\mathbf{w}^\intercal\mathbf{x}=\begin{bmatrix}w_1 & \cdots & w_n\end{bmatrix}\begin{bmatrix}x_1 \\ \vdots \\ x_n\end{bmatrix}=w_1x_1+\cdots+w_nx_n
\label{eq:linear}\tag{3.3}
\end{equation}

Let $t$ be the threshold that separates classes in our projected space. For an input vector $\mathbf{x}$, if the projected value $y\geq t$ then $\mathbf{x}$ belongs to class $G$, otherwise $\mathbf{x}$ belongs to class $B$. 

We would like to maximize the separability of projected points, $y$, based on Fisher's first criteria. In order to do this, we must first define a measure of separation between different classes. Let $m_i$ represent the mean of the series of vectors belonging to class $C_i$. For $m$-dimensional vectors $\mathbf{x}$ that belong to $C_i$, we say $\mathbf{x}\in C_i$. Finally, $N_i$ represents the number of vectors belonging to class $C_i$:

\begin{equation}
m_i=\frac{1}{N_i}\sum_{\mathbf{x}\in C_i}{\mathbf{x}}
\label{eq:real_means}\tag{3.4}
\end{equation}

Recall, \eqref{eq:fisher} requires difference of per-class *projected* means. Let $\hat{m_i}$ represent the mean vector for each class projected onto a boundary:

\begin{equation}
\hat{m_i}=\frac{1}{N_i}\sum_{y\in C_i}{y}=\frac{1}{N_i}\sum_{\mathbf{x}\in C_i}{\mathbf{w}^\intercal\mathbf{x}}=\mathbf{w}^\intercal\left(\frac{1}{N_i}\sum_{\mathbf{x}\in C_i}{\mathbf{x}}\right)=\mathbf{w}^\intercal m_i
\label{eq:projected_means}\tag{3.5}
\end{equation}

Notice, the projected mean vector, $\hat{m_i}$, is simply a linear combination of our vector normal to the hyperplane and the original mean vector value, $m_i$. The numerator of \eqref{eq:fisher} is the squared difference between projected class mean values. Why are we squaring the difference? Spatially, the difference between mean values is a scalar distance, meaning $\hat{m_G}-\hat{m_B}$ is the distance between projected mean values of our two classes. Since we don't know if $\hat{m_G}\gt\hat{m_B}$, or if $\hat{m_G}\lt\hat{m_B}$, we square the mean value so the distance will always result in a positive number. Intuitively, this makes sense, since a negative distance is not possible.

The second condition of Fisher's Linear Discriminant function aims to minimize within-class variance, or scatter, defined by Fisher's second criteria. Let $\hat{s_i}$ represent the within-class variance for class $C_i$ in the projected space. The variance is the sum of square differences between values and their classes mean value. Consider the variability of within-class vectors after projection onto the $y$-space:

\begin{equation}
\hat{s_i}=\sum_{y\in C_i}{(y-\hat{m_i})^2}
\label{eq:projected_scatter}\tag{3.6}
\end{equation}

In our two-class example, $\hat{s_G}+\hat{s_B}$ measures the variability within two classes *after* projection onto the hyperplane. We have successfully defined our projected values to satisfy the numerator and denominator of \eqref{eq:fisher}, however, in order to find a maximized projection we need to express $J(\mathbf{w})$ as a function of our input vectors. We can substitute values for $y$ and $\hat{m_i}$ to find $\hat{s_i}$ in terms of input vectors:

\begin{equation}
\hat{s_i}=\sum_{y\in C_i}{(y-\hat{m_i})^2}=\sum_{\mathbf{x}\in C_i}{(\mathbf{w}^\intercal\mathbf{x}-\mathbf{w}^\intercal m_i)^2}=\mathbf{w}^\intercal\left(\sum_{\mathbf{x}\in C_i}{(\mathbf{x}-m_i)(\mathbf{x}-m_i)^\intercal}\right)\mathbf{w}
\label{eq:real_scatter}\tag{3.7}
\end{equation}

Before we transform from the projected space back to the original space, let us define $S_i$ as the **covariance matrix**, or **scatter matrix**, for class $C_i$:

\begin{equation}
S_i=\left(\sum_{\mathbf{\mathbf{x}}\in C_i}{(\mathbf{x}-m_i)(\mathbf{x}-m_i)^\intercal}\right)
\label{eq:scatter_matrix}\tag{3.8}
\end{equation}

The scatter matrix will tell us the variance, or separability of points within a class, and also the separability of means between classes. Using the definition $\hat{s_G}+\hat{s_B}$ as the within-class variance after projection, the within-class scatter matrix *before* projection, $S_w$, for classes $G$, and $B$ is $S_w=S_G+S_B$:

\begin{equation}
\hat{s_G}+\hat{s_B}=\mathbf{w}^\intercal S_G\mathbf{w}+\mathbf{w}^\intercal S_B\mathbf{w}=\mathbf{w}^\intercal S_w\mathbf{w}=\hat{S_w}
\label{eq:variability_two_class}\tag{3.9}
\end{equation}

Where $\hat{S_w}$ is the within-class scatter matrix of projected values $y$, and $S_w$ is the within-class scatter matrix of the original feature vectors. Using the same logic, we can express the difference of between-class means before projection using the scatter matrix $S_m$:

\begin{equation}
(\hat{m_G}-\hat{m_B})^2=\mathbf{w}^\intercal(m_G-m_B)(m_G-m_B)^\intercal\mathbf{w}=\mathbf{w}^\intercal S_m\mathbf{w}=\hat{S_m}
\label{eq:real_means_squared}\tag{3.10}
\end{equation}

Where $\hat{S_m}$ is the between-class scatter matrix of the projected values $y$, and $S_m$ is the between-class scatter matrix of the original feature vectors. We can now re-write \eqref{eq:fisher} in terms of our original feature space:

\begin{equation}
J(\mathbf{w})=\frac{(\hat{m_a}-\hat{m_b})^2}{\hat{s_a}+\hat{s_b}}=\frac{\mathbf{w}^\intercal S_m\mathbf{w}}{\mathbf{w}^\intercal S_w\mathbf{w}}
\label{eq:fisher_real}\tag{3.11}
\end{equation}

In order to find the maximum of $J(\mathbf{w})$, we can differentiate and equate it to zero.

It is worth noting that from our previously found "best" projection we can run the same optimization again with the constraint that the new "best" projection must be orthogonal to the previous one. The new orthogonal vector will serve as a basis vector for all vectors in the reduced vector space. This is useful as a form of dimensionality reduction, and our first exposure to unsupervised learning.

For classification problems with more than two classes, there are minor extensions to be made. First, we can take the difference between the class mean and "global" mean (the mean-value for all vectors in that space), and scale by the number of feature vectors in that particular class:

\begin{equation}
S_m=\sum_{i=1}^k{N_i(m_i-m)(m_i-m)^\intercal}
\label{eq:multi_mean}\tag{3.12}
\end{equation}

Where $k$ is the number of classes, $N_i$ is the number of points in a given class, $m_i$ is the mean of the class, and $m$ is the global mean for all vectors in the space. Next, we can find the within-class scatter for each class:

\begin{equation}
S_w=\sum_{i=1}^k{S_i}=\sum_{i=1}^k\sum_{\mathbf{x}\in C_i}{(\mathbf{x}-m_i)(\mathbf{x}-m_i)^\intercal}
\label{eq:multi_scatter}\tag{3.13}
\end{equation}

Finally, project onto a lower dimensional space, using the normal vector $\mathbf{w}$. maximize between-class variance and minimizes within-class variance:

\begin{equation}
J(\mathbf{w})=\mathop{\mathrm{argmax}}\frac{\mathbf{w}^\intercal S_m\mathbf{w}}{\mathbf{w}^\intercal S_w\mathbf{w}}
\label{eq:argmax}\tag{3.14}
\end{equation}

Where $\mathop{\mathrm{argmax}}$ finds the maximum value of the objective function.

In practice, rather than solving for one vector at a time iteratively, you should solve for the entire **orthonormal basis** simultaneously. This is accomplished by applying Singular Value Decomposition to a matrix, which is the product of the inverse-within-class-scatter matrix and the between-class-scatter matrix. The **eigenvectors** of that product form the new basis vectors, and the corresponding **eigenvalues** tell you the importance of the basis vector. I will omit the mathematical explanation for this, but I encourage you to explore how it is done on your own.

### Algorithm

Here is a general algorithm for LDA:

1. Calculate the separability of between-class means, or the distance between the mean of different classes. To do this, find the between-class variance using equation \eqref{eq:multi_mean} above.
2. Calculate the within-class variance for each class using equation \eqref{eq:multi_scatter} above.
3. Project onto a lower dimensional space using equation \eqref{eq:argmax}.

### In Action

Let's explore LDA using the SciKit Learn model definition of [Linear Discriminant Analysis](https://scikit-learn.org/stable/modules/generated/sklearn.discriminant_analysis.LinearDiscriminantAnalysis.html)

TODO: Finish this section. Incorporate Elsevier specific data?

## Support Vector Machines

TODO: Still needs to be finished...

Another approach for handling hyperplane generation is to find a boundary with the maximum **margin**, which is to find the distance from a boundary to the closest data point on either side of it. Intuitively, we want a boundary that's nicely centered between the classes, because it is equidistant from the nearest known examples of both of them and will improve the hypothesis space for new points. The hyperplane that maximizes the margin is known as an **optimal separating hyperplane**. There is a key distinction that must be made by this definition: The optimal separating hyperplane guarantees optimality with respect to the margin, not necessarily with respect to classification accuracy. We refer to the points nearest to the boundary as **support vectors**. These points are critical towards optimizing the boundary location of the hyperplane, as they are the limit values used to separate classes.

What happens when there is no clean way to segregate points of different classes? In real examples, we rarely find data that is so cleanly separated that drawing hyperplane lines is trivial.

How to use SVM for regression?

SVM is commonly seen in text classification problems, such as sentiment analysis or spam filtering, etc...

### Mathematical Intuition

<img src="../img/3_svm.png" alt="https://en.wikipedia.org/wiki/Support_vector_machine" style="width: 300px;"/>